## Text noun phrase classifier

In [1]:
import jsonlines
import tqdm
import json
import spacy
import numpy as np 
import pandas as pd
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# import scispacy
import spacy
# import en_core_web_sm

from scipy.spatial.distance import jensenshannon

import joblib

from IPython.display import HTML, display

from ipywidgets import interact, Layout, HBox, VBox, Box
import ipywidgets as widgets
from IPython.display import clear_output

from tqdm import tqdm
from os.path import isfile

import seaborn as sb
import matplotlib.pyplot as plt
plt.style.use("dark_background")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
from nltk.tokenize import word_tokenize

from multiprocessing import Pool

import os, re, csv, math, codecs
from sklearn.model_selection import train_test_split
sns.set_style("whitegrid")
np.random.seed(0)

os.environ["CUDA_VISIBLE_DEVICES"]="1"

stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])
# DATA_PATH = '/data/szr207/projects/log_analysis/equal_scidocs.csv'

from subprocess import check_output
# print(check_output(["ls", DATA_PATH]).decode("utf8"))
# plt.style.use("dark_background")

# DATA_PATH = '/data/szr207/dataset/scidocs/mag/combined.csv'


2023-01-29 17:57:33.563646: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-01-29 17:57:35.507963: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-29 17:57:35.508069: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-29 17:57:35.508608: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1d:00.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.77GiB deviceMemoryBandwidth: 836.37GiB/s
2023-01-29 17:57:35.508636: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-01-29 17:57:35.510989: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2

- number of tokens
- number of capital letters
- number of special characters
- consecutive punctuation marks
- percent of stop-words in text 
- min document freq
- max DF
- median document freq

In [2]:
df_data = pd.read_csv('noun_title_classification.csv')

In [3]:
df_data['doc_len'] = df_data['text'].apply(lambda words: len(word_tokenize(str(words))))


max_seq_len = 41
sns.distplot(df_data['doc_len'], hist=True, kde=True, color='b', label='doc len')
plt.axvline(x=max_seq_len, color='k', linestyle='--', label='max len')
plt.title('comment length'); plt.legend()
plt.show()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  """
/opt/conda/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1402: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  ndim = x[:, None].ndim
/opt/conda/lib/python3.7/site-packages/matplotlib/axes/_base.py:276: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.n

In [4]:
# number of capital letters

def get_capital_count(x):
    count = 0
    for i in word_tokenize(str(x)):
        if i.isupper():
            count+=1
    return count

In [5]:
df_data['upper_count'] = df_data['text'].apply(lambda x:get_capital_count(x))

In [6]:
df_data

,text,label,doc_len,upper_count
0,they,np,1,0
1,PhD training in simulation: NATCOR,title,6,1
2,Effect of SLCO1B1 Polymorphisms on Rifabutin P...,title,14,1
3,methodology,np,1,0
4,A randomized controlled trial of the efficacy ...,title,34,1
...,...,...,...,...
96941,Profitability of Canadian- Versus U.S.-control...,title,6,0
96942,neutropenia,np,1,0
96943,"Multidisciplinary Essays. (Book Reviews: Mind,...",title,25,0
96944,Major bleeding,np,2,0


In [7]:
df_data['upper_count']

0        0
1        1
2        1
3        0
4        1
        ..
96941    0
96942    0
96943    0
96944    0
96945    0
Name: upper_count, Length: 96946, dtype: int64

In [8]:
from string import punctuation

def get_punctuation_counts(x):
    punctuation_counts = 0
    for i in word_tokenize(str(x)):
        if i in punctuation:
            punctuation_counts+=1
    return punctuation_counts

In [9]:
def stop_word_count(x):
    stop_count = 0
    for i in word_tokenize(str(x)):
        if i in stop_words:
            stop_count+=1
    return stop_count

In [10]:
get_punctuation_counts('PhD training in simulation: NATCOR')

1

In [11]:
df_data

,text,label,doc_len,upper_count
0,they,np,1,0
1,PhD training in simulation: NATCOR,title,6,1
2,Effect of SLCO1B1 Polymorphisms on Rifabutin P...,title,14,1
3,methodology,np,1,0
4,A randomized controlled trial of the efficacy ...,title,34,1
...,...,...,...,...
96941,Profitability of Canadian- Versus U.S.-control...,title,6,0
96942,neutropenia,np,1,0
96943,"Multidisciplinary Essays. (Book Reviews: Mind,...",title,25,0
96944,Major bleeding,np,2,0


In [12]:
df_data['punct_count'] = df_data['text'].apply(lambda x:get_punctuation_counts(x))

In [13]:
df_data['stop_count'] = df_data['text'].apply(lambda x:stop_word_count(x))

In [14]:
df_data = df_data.dropna()

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

path_sci = 'sci_dump.txt'

with open(path_sci) as f:
    text_lines = f.read().splitlines()

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=50000)
X = vectorizer.fit_transform(text_lines)
# print(vectorizer.get_feature_names())

# print(X.shape)

In [17]:
df_data = df_data.dropna().reset_index(drop=True)

In [18]:
tf_idf = vectorizer.transform(df_data['text'])

In [19]:
df_data.shape

(96944, 6)

In [20]:
a = tf_idf[1].toarray()[0]

In [21]:
import numpy as np

minval = np.min(a[np.nonzero(a)])
maxval = np.max(a[np.nonzero(a)])
median = np.median(a[np.nonzero(a)])

In [22]:
maxval

0.7409509708429297

In [23]:
median

0.46856287260010354

In [24]:
minval

0.10559593437062544

In [25]:
min_list = []
max_list = []
med_list = []

for i in tqdm(tf_idf):
    a = i.toarray()[0]
    try:
        min_list.append(np.min(a[np.nonzero(a)]))
        max_list.append(np.max(a[np.nonzero(a)]))
        med_list.append(np.median(a[np.nonzero(a)]))
    except:
        min_list.append(0.0)
        max_list.append(0.0)
        med_list.append(0.0)

96944it [02:48, 576.33it/s]


In [26]:
max_list

[1.0,
 0.7409509708429297,
 0.4454618668273191,
 1.0,
 0.3808892603509144,
 0.506178997810789,
 0.37918548619103043,
 0.5209875467452301,
 1.0,
 0.4105259083571916,
 0.4553198673516652,
 0.9877428014306799,
 0.4063911808920798,
 0.7349530368142965,
 1.0,
 0.5022559557523124,
 0.42586952215583757,
 1.0,
 0.46281691029153726,
 1.0,
 1.0,
 0.6040228295559257,
 1.0,
 0.6945391010772781,
 0.5457375750489417,
 0.4152443250124351,
 0.5234370751010409,
 0.5148706058989336,
 1.0,
 0.4189216052878228,
 0.776445338477066,
 1.0,
 0.5265585890989474,
 0.7619097582645955,
 0.8727874139322936,
 0.8314622213679533,
 1.0,
 0.5435898335896531,
 0.6581636320067723,
 0.7266266923791006,
 1.0,
 0.5408300022139203,
 0.48226327702706295,
 1.0,
 0.5686814160641912,
 0.4889343747996727,
 1.0,
 1.0,
 0.7634582840709473,
 0.6986393347515628,
 0.41003769920174454,
 0.796849186766923,
 0.6371707132389685,
 0.7972653621635752,
 0.5886735344347002,
 0.41681809694995486,
 0.6565781444546153,
 1.0,
 0.5097914373625645

In [27]:
df_min.shape

NameError: name 'df_min' is not defined

In [28]:
df_min = pd.Series(min_list,
              name="min")
df_min = df_min.to_frame()

df_max = pd.Series(max_list,
              name="max")
df_max = df_max.to_frame()

df_median = pd.Series(med_list,
              name="median")
df_median = df_median.to_frame()

In [29]:
df_median

,median
0,1.000000
1,0.468563
2,0.260455
3,1.000000
4,0.158462
...,...
96939,0.439050
96940,1.000000
96941,0.228789
96942,0.670377


In [30]:
df_final = pd.concat([df_data, df_min, df_max, df_median], axis=1)

In [31]:
df_final

,text,label,doc_len,upper_count,punct_count,stop_count,min,max,median
0,they,np,1,0,0,1,1.000000,1.000000,1.000000
1,PhD training in simulation: NATCOR,title,6,1,1,2,0.105596,0.740951,0.468563
2,Effect of SLCO1B1 Polymorphisms on Rifabutin P...,title,14,1,1,5,0.055986,0.445462,0.260455
3,methodology,np,1,0,0,0,1.000000,1.000000,1.000000
4,A randomized controlled trial of the efficacy ...,title,34,1,1,8,0.025805,0.380889,0.158462
...,...,...,...,...,...,...,...,...,...
96939,Profitability of Canadian- Versus U.S.-control...,title,6,0,0,1,0.076646,0.509872,0.439050
96940,neutropenia,np,1,0,0,0,1.000000,1.000000,1.000000
96941,"Multidisciplinary Essays. (Book Reviews: Mind,...",title,25,0,7,12,0.045121,0.556299,0.228789
96942,Major bleeding,np,2,0,0,0,0.445445,0.895309,0.670377


In [32]:
normalized_df_len=(df_final['doc_len']-df_final['doc_len'].min())/(df_final['doc_len'].max()-df_final['doc_len'].min())

In [33]:
normalized_df_upper_count=(df_final['upper_count']-df_final['upper_count'].min())/(df_final['upper_count'].max()-df_final['upper_count'].min())

In [34]:
normalized_df_punct_count=(df_final['punct_count']-df_final['punct_count'].min())/(df_final['punct_count'].max()-df_final['punct_count'].min())

In [35]:
normalized_df_stop_count=(df_final['stop_count']-df_final['stop_count'].min())/(df_final['stop_count'].max()-df_final['stop_count'].min())

In [36]:
df = pd.concat([normalized_df_len, normalized_df_upper_count, normalized_df_punct_count, normalized_df_stop_count,df_final['min'].to_frame(), df_final['max'].to_frame(), df_final['median'].to_frame() ], axis=1)

In [37]:
#df_features.to_csv('features.csv')

NameError: name 'df_features' is not defined

In [38]:
df_final['label'].to_frame().to_csv('label.csv')

In [39]:
df.loc[:,~df.columns.duplicated()].to_csv('features.csv', index= False)

In [40]:
df.to_csv('features.csv', index= False)

##  Test data feature extraction

In [2]:
data_pred = pd.read_csv('etd_records_v5.csv')
data_pred

,id,title,author,year,university,degree,advisor,department
0,2209,"Urban Form, Wind, Comfort, and Sustainability:...","Kim, Hyungkyoo",2014,"University of California, Berkeley",Doctor Of Philosophy,"Macdonald, Elizabeth;",City And Regional Planning
1,2294,"Movement, Abundance Patterns, and Foraging Eco...","Hofmeister, Jennifer Krista Kaulalani",2015,"University of California, Berkeley",Doctor Of Philosophy,"Caldwell, Roy L;",NaN
2,3735,Regulation of Form in Multicellular Choanoflag...,"Larson, Benjamin Timothy",2019,"University of California, Berkeley",Doctor Of Philosophy,"King, Nicole;Hallatschek, Oskar;",Computational And Genomic Biology
3,4255,Attacks on Emerging Architectures,"Hanna, Steven Craig",2012,"University of California, Berkeley",Doctor Of Philosophy,"Song, Dawn;",Computer Science
4,4485,Study of Laser Melted Liquids and of Interfaci...,"Hull, Christopher James",2019,"University of California, Berkeley",Doctor Of Philosophy,"Saykally, Rich;",Chemistry
...,...,...,...,...,...,...,...,...
494,279281,Metaphor: A communication tool,"Patel, Alpesh R.",NaN,Texas Tech University,Bachelor of Architecture,NaN,Architecture
495,320921,The rationalization of a system for the manufa...,"Eflin, Robert Dean",NaN,Rice University,Master of Architecture,NaN,Architecture
496,277909,The Lubbock Daily Times,"Van Dine, William A. Jr.",NaN,Texas Tech University,Bachelor of Architecture,NaN,Architecture
497,275021,Altruistic civic architecture through selected...,"Ritchie, Blaik",NaN,Texas Tech University,Bachelor of Architecture,NaN,Architecture


In [3]:
data_pred = data_pred.drop(['id', 'author', 'author', 'year', 'university', 'degree', 'advisor','department'], axis=1)

In [4]:
data_pred

,title
0,"Urban Form, Wind, Comfort, and Sustainability:..."
1,"Movement, Abundance Patterns, and Foraging Eco..."
2,Regulation of Form in Multicellular Choanoflag...
3,Attacks on Emerging Architectures
4,Study of Laser Melted Liquids and of Interfaci...
...,...
494,Metaphor: A communication tool
495,The rationalization of a system for the manufa...
496,The Lubbock Daily Times
497,Altruistic civic architecture through selected...


In [5]:
data_pred.to_csv('titles.csv', index = None)

In [78]:
data_pred['label'] = np.nan
data_pred = data_pred.fillna("title")

In [81]:
data_pred = data_pred.rename(columns={"title": "text"})
data_pred['label'].to_frame().to_csv('label_pred.csv')

In [50]:
tqdm.pandas()

In [51]:
data_pred['doc_len'] = data_pred['text'].apply(lambda words: len(word_tokenize(str(words))))

In [52]:
data_pred = data_pred.dropna().reset_index(drop=True)

In [53]:
def capital_apply_df(df):
    df['upper_count'] = df['text'].apply(lambda x:get_capital_count(x))
    return df

In [61]:
def punct_count_apply_df(df):
    df['punct_count'] = df['text'].apply(lambda x:get_punctuation_counts(x))
    return df

def parallelize_dataframe(df, func, n_cores=40):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [62]:
def stop_word_count_apply_df(df):
    df['stop_count'] = df['text'].apply(lambda x:stop_word_count(x))
    return df

In [63]:
data_pred = parallelize_dataframe(data_pred, capital_apply_df)

In [64]:
data_pred = parallelize_dataframe(data_pred, punct_count_apply_df)

In [65]:
data_pred = parallelize_dataframe(data_pred, stop_word_count_apply_df)

In [66]:
data_pred

,text,doc_len,upper_count,punct_count,stop_count
0,"Urban Form, Wind, Comfort, and Sustainability:...",14,0,4,5
1,"Movement, Abundance Patterns, and Foraging Eco...",17,0,3,6
2,Regulation of Form in Multicellular Choanoflag...,13,0,0,5
3,Attacks on Emerging Architectures,4,0,0,1
4,Study of Laser Melted Liquids and of Interfaci...,12,0,0,4
...,...,...,...,...,...
494,Metaphor: A communication tool,5,1,1,1
495,The rationalization of a system for the manufa...,13,0,0,6
496,The Lubbock Daily Times,4,0,0,0
497,Altruistic civic architecture through selected...,7,0,0,1


In [67]:
tf_idf = vectorizer.transform(data_pred['text'])

In [68]:
min_list = []
max_list = []
med_list = []

for i in tqdm(tf_idf):
    a = i.toarray()[0]
    try:
        min_list.append(np.min(a[np.nonzero(a)]))
        max_list.append(np.max(a[np.nonzero(a)]))
        med_list.append(np.median(a[np.nonzero(a)]))
    except:
        min_list.append(0.0)
        max_list.append(0.0)
        med_list.append(0.0)

499it [00:00, 559.13it/s]


In [69]:
df_min = pd.Series(min_list,
              name="min")
df_min = df_min.to_frame()

df_max = pd.Series(max_list,
              name="max")
df_max = df_max.to_frame()

df_median = pd.Series(med_list,
              name="median")
df_median = df_median.to_frame()

In [70]:
normalized_df_len=(data_pred['doc_len']-data_pred['doc_len'].min())/(data_pred['doc_len'].max()-data_pred['doc_len'].min())

normalized_df_upper_count=(data_pred['upper_count']-data_pred['upper_count'].min())/(data_pred['upper_count'].max()-data_pred['upper_count'].min())

normalized_df_punct_count=(data_pred['punct_count']-data_pred['punct_count'].min())/(data_pred['punct_count'].max()-data_pred['punct_count'].min())
normalized_df_stop_count=(data_pred['stop_count']-data_pred['stop_count'].min())/(data_pred['stop_count'].max()-data_pred['stop_count'].min())

In [71]:
df_pred_final = pd.concat([normalized_df_len, normalized_df_upper_count, normalized_df_punct_count, normalized_df_stop_count,df_min['min'].to_frame(), df_max['max'].to_frame(), df_median['median'].to_frame() ], axis=1)

In [72]:
df_pred_final.to_csv('pred_features.csv', index=False)

In [73]:
!head pred_features.csv

doc_len,upper_count,punct_count,stop_count,min,max,median
0.32432432432432434,0.0,0.6666666666666666,0.35714285714285715,0.06064969578958504,0.44252716138738984,0.32558457101839633
0.40540540540540543,0.0,0.5,0.42857142857142855,0.03765046824132918,0.7809246595140266,0.19920419749895057
0.2972972972972973,0.0,0.0,0.35714285714285715,0.05752577915886868,0.5354872137688079,0.2392530356641635
0.05405405405405406,0.0,0.0,0.07142857142857142,0.13634984473142972,0.6395652560384795,0.5321135586624239
0.2702702702702703,0.0,0.0,0.2857142857142857,0.05922407362929875,0.4725474316413063,0.31414328209608916
0.6486486486486487,0.05,0.16666666666666666,0.35714285714285715,0.02710414525977461,0.6336835281860314,0.15143355510593806
0.2972972972972973,0.0,0.0,0.2857142857142857,0.04838580728176038,0.46095945268815647,0.3030301159630567
0.21621621621621623,0.0,0.0,0.14285714285714285,0.06702545356451044,0.5407048765022479,0.3200914875842151
0.35135135135135137,0.0,0.0,0.2857142857142857,0.0427264053806

In [74]:
data_pred

,text,doc_len,upper_count,punct_count,stop_count
0,"Urban Form, Wind, Comfort, and Sustainability:...",14,0,4,5
1,"Movement, Abundance Patterns, and Foraging Eco...",17,0,3,6
2,Regulation of Form in Multicellular Choanoflag...,13,0,0,5
3,Attacks on Emerging Architectures,4,0,0,1
4,Study of Laser Melted Liquids and of Interfaci...,12,0,0,4
...,...,...,...,...,...
494,Metaphor: A communication tool,5,1,1,1
495,The rationalization of a system for the manufa...,13,0,0,6
496,The Lubbock Daily Times,4,0,0,0
497,Altruistic civic architecture through selected...,7,0,0,1
